In [13]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.support.ui import WebDriverWait
import bs4
import pandas as pd
import re
import time


In [2]:
driver_path = 'C:/Users/jaehkim/Desktop/sermon_analysis/chromedriver.exe'
url = 'https://www.fgtv.com/fgtv/f1/01.asp'

driver = webdriver.Chrome(driver_path)
driver.implicitly_wait(3)

driver.get(url)

# FNGTV 방송 - 예배 0~9 Loop

In [8]:
# left_menu_titles : 한 번 돌아갈떄마다 바뀌므로 동시에 실행
left_menu_titles = driver.find_element_by_class_name("depth2_gap")

worships = left_menu_titles.text.split('\n') #categories
buttons = left_menu_titles.find_elements_by_tag_name('a') 
for button in buttons:
    button.click()
    driver.implicitly_wait(3)

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=84.0.4147.105)


# 1부~7부 Loop

In [17]:
mainframe = driver.find_element_by_class_name("mainframe")
worship_times = mainframe.find_elements_by_tag_name('li')

for worship_time in worship_times:
    if re.search('\d부',worship_time.text):
        print(worship_time.text)
        #driver.implicitly_wait(3) 시도(1)
        #time.sleep(5) #시도(2)
        #wait = WebDriverWait(driver, 20) #시도(3)
        #element=wait.until(EC.presence_of_element_located((By.ID,worship_time.id)))
        #element.click()
    

1부
2부
3부
4부
5부(청년연합예배)
6부
7부


# Table

In [53]:
tables = driver.find_elements_by_class_name('table_f1')
table_columns = tables[0].text.split()

In [92]:
table_columns 

['날짜', '설교자', '제목', '본문', '동영상', '선택']

In [74]:
row_element = tables[1].find_elements_by_tag_name('td')

In [110]:
row,rows = [],[]
for idx,x in enumerate(row_element):
    if idx%7 == 6:
        rows.append(row)
        row = []
        continue
    print(idx,x.text)
    row.append(x.text)


0 2020-08-02
1 이장균 목사
2 믿음으로 에녹은
3 히 11:5
4 고화질
5 
7 2020-07-26
8 이장균 목사
9 믿음으로 아벨은
10 히 11:4
11 고화질
12 
14 2020-07-19
15 이영훈 목사
16 주님께서 나를 구원하시리로다
17 시 55:16~18, 22
18 고화질
19 
21 2020-07-12
22 이영훈 목사
23 성령충만의 4단계
24 겔 47:1~5
25 고화질
26 
28 2020-07-05
29 이영훈 목사
30 광야길 40년
31 출 14:13~14
32 고화질
33 
35 2020-06-28
36 이영훈 목사
37 주님을 사랑하라
38 시 31:23~24
39 고화질
40 
42 2020-06-21
43 이영훈 목사
44 상한 갈대를 꺾지 아니하며
45 마 12:17~21
46 고화질
47 
49 2020-06-14
50 이영훈 목사
51 날마다 우리 짐을 지시는 주
52 시 68:19~21
53 고화질
54 
56 2020-06-07
57 이영훈 목사
58 믿음이 작은 자여 왜 의심하였느냐?
59 마 14:24~32
60 고화질
61 
63 2020-05-31
64 이영훈 목사
65 성령의 시대
66 행 2:17~18
67 고화질
68 


In [112]:
pd.DataFrame(data=rows,columns=table_columns)

,날짜,설교자,제목,본문,동영상,선택
0,2020-08-02,이장균 목사,믿음으로 에녹은,히 11:5,고화질,
1,2020-07-26,이장균 목사,믿음으로 아벨은,히 11:4,고화질,
2,2020-07-19,이영훈 목사,주님께서 나를 구원하시리로다,"시 55:16~18, 22",고화질,
3,2020-07-12,이영훈 목사,성령충만의 4단계,겔 47:1~5,고화질,
4,2020-07-05,이영훈 목사,광야길 40년,출 14:13~14,고화질,
5,2020-06-28,이영훈 목사,주님을 사랑하라,시 31:23~24,고화질,
6,2020-06-21,이영훈 목사,상한 갈대를 꺾지 아니하며,마 12:17~21,고화질,
7,2020-06-14,이영훈 목사,날마다 우리 짐을 지시는 주,시 68:19~21,고화질,
8,2020-06-07,이영훈 목사,믿음이 작은 자여 왜 의심하였느냐?,마 14:24~32,고화질,
9,2020-05-31,이영훈 목사,성령의 시대,행 2:17~18,고화질,


# 페이지 넘기기